<center><img src='../assets/images/aau_logo.png' width='10%'></center>

<br>
<center><h3>Addis Ababa University
<br>College of Natural and Computational Sciences
<br>Department of Computer Science</h3></center>

---
<center><h1 style="font-weight: bold;">CoSc 6262: Data Science</h1></center>
<center><img src='../assets/images/scipy_logo.png' width='15%'></center>
<br>
<center><h2>Assignment 4: SciPy II</h2> </center>

---

## Questions

### Part 1: Sparse Matrices

#### Question 1

Create two random sparse matrices, x and y. The first one must have 1000 rows and 1000 columns and should contain the value 0.99 along the diagonal and 0 everywhere else. The second one should have 1000 rows and 500 columns. Assign random numbers between 50 and 100 to 20% of the values within this matrix. The remaining 80% of the values should be zero. (Hint: look at the scipy.sparse.rand function)

Then do a matrix multiplication of x and y, and store the result in a new variable, z

In [1]:
import numpy as np
from scipy.sparse import diags, random as sparse_random

# Create sparse matrix x of shape (1000, 1000) with 0.99 along the diagonal
x = diags([0.99], [0], shape=(1000, 1000), format='csr')

# Create sparse matrix y of shape (1000, 500) with 20% non-zero entries and values between 50 and 100
rvs = lambda s: np.random.uniform(50, 100, size=s)
y = sparse_random(1000, 500, density=0.2, data_rvs=rvs, format='csr')

# Multiply x and y, store the result in z
z = x.dot(y)

print("Matrix x shape:", x.shape)
print("Matrix y shape:", y.shape)
print("Matrix z shape:", z.shape)

Matrix x shape: (1000, 1000)
Matrix y shape: (1000, 500)
Matrix z shape: (1000, 500)


#### Question 2

Find out how many rows and columns are in the new matrix z, calculted in question 1. Is it what you would expect?

In [2]:
# z is the product of a (1000 x 1000) and a (1000 x 500) matrix.
print("Shape of matrix z: ", z.shape)
print("Expected shape: (1000, 500)")

Shape of matrix z:  (1000, 500)
Expected shape: (1000, 500)


#### Question 3

Calculate the determinant and Frobenius norm of z

In [3]:
try:
    det_z = np.linalg.det(z.toarray())
    print("Determinant of z:", det_z)
except Exception as e:
    print("Determinant could not be computed (matrix is not square):", e)

# Compute the Frobenius norm of z
fro_norm = np.linalg.norm(z.toarray(), 'fro')
print("Frobenius norm of z:", fro_norm)

Determinant could not be computed (matrix is not square): Last 2 dimensions of the array must be square
Frobenius norm of z: 23894.860115867807


#### Question 4

Create a matrix with 1000 rows and 1500 columns by joining the matrices x and y at the side into one matrix

In [4]:
from scipy.sparse import hstack

# Horizontally stack x and y
joined_matrix = hstack([x, y], format='csr')
print("Joined matrix shape:", joined_matrix.shape)
print("Expected shape: (1000, 1500)")

Joined matrix shape: (1000, 1500)
Expected shape: (1000, 1500)


> The output of the print statements confirms that matrix z has the expected shape, (1000, 500). This is exactly what you'd expect from multiplying a (1000 x 1000) matrix with a (1000 x 500) matrix.

### Part 2: Stats in Scipy

In [5]:
from scipy import stats
import numpy as np

#### Question 5

Create two samples of 1000 numbers each. The first should be from a standard normal distribution (i.e. mean 0 and standard deviation of 1) and the second should be from a normal distribution with mean 10 and standard distribution 3.

Do a t-test on these two datasets. Is the result what you would expect?

In [6]:
from scipy import stats
import numpy as np
np.random.seed(42)

# Create two samples of 1000 numbers each
sample1 = np.random.normal(0, 1, 1000)       # standard normal (mean=0, std=1)
sample2 = np.random.normal(10, 3, 1000)      # normal with mean=10 and std=3

# Perform an independent t-test
t_stat, p_value = stats.ttest_ind(sample1, sample2)
print("T-test result:")
print("t-statistic =", t_stat)
print("p-value =", p_value)

T-test result:
t-statistic = -102.37756272192493
p-value = 0.0


> Since the two samples are drawn from distributions with very different means and different variances, the t-test produces a very large t-statistic and a near-zero p-value, indicating a statistically significant difference between the two groups.

#### Question 6

Use Plotly to plot histograms of the two distributions you just created. The histograms for each should be on the same plot.

In [7]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=sample1,
    opacity=0.75,
    name='Standard Normal'
))

fig.add_trace(go.Histogram(
    x=sample2,
    opacity=0.75,
    name='Normal (mean=10, std=3)'
))

# Use overlay mode for the two histograms
fig.update_layout(
    barmode='overlay',
    title_text='Histograms of Two Normal Distributions',
    xaxis_title_text='Value',
    yaxis_title_text='Count'
)

fig.show()

## Additional Cells

### Density heatmap of matrix y

In [8]:
import plotly.express as px

# Extract row and column indices of non-zero entries
rows, cols = y.nonzero()

# Create a density heatmap (2D histogram) of non-zero entries
fig_density = px.density_heatmap(x=cols, y=rows, nbinsx=50, nbinsy=50,
                                 labels={"x": "Column Index", "y": "Row Index"},
                                 title="Density Heatmap of Non-Zero Entries in Matrix y")
fig_density.show()

### Kernel density estimation

In [9]:
from scipy.stats import gaussian_kde
import numpy as np
import plotly.graph_objects as go

# Create a range of x values covering both distributions
x_range = np.linspace(min(sample1.min(), sample2.min()) - 1,
                      max(sample1.max(), sample2.max()) + 1, 1000)

# Compute KDE for both samples
kde1 = gaussian_kde(sample1)
kde2 = gaussian_kde(sample2)

density1 = kde1(x_range)
density2 = kde2(x_range)

fig_kde = go.Figure()

fig_kde.add_trace(go.Scatter(
    x=x_range,
    y=density1,
    mode='lines',
    name='KDE Standard Normal'
))

fig_kde.add_trace(go.Scatter(
    x=x_range,
    y=density2,
    mode='lines',
    name='KDE Normal (mean=10, std=3)'
))

fig_kde.update_layout(
    title='Kernel Density Estimation of Two Distributions',
    xaxis_title='Value',
    yaxis_title='Density'
)

fig_kde.show()

### Q-Q plots for the two distributions

In [10]:
import scipy.stats as st
import plotly.graph_objects as go

# Compute Q-Q plot data for sample1 (standard normal)
qq1 = st.probplot(sample1, dist="norm", plot=None)
theoretical_q1, sample_q1 = qq1[0]

# Compute Q-Q plot data for sample2 (normal with mean=10, std=3)
qq2 = st.probplot(sample2, dist="norm", plot=None)
theoretical_q2, sample_q2 = qq2[0]

fig_qq = go.Figure()

fig_qq.add_trace(go.Scatter(
    x=theoretical_q1,
    y=sample_q1,
    mode='markers',
    name='Standard Normal QQ'
))
fig_qq.add_trace(go.Scatter(
    x=theoretical_q2,
    y=sample_q2,
    mode='markers',
    name='Normal (mean=10, std=3) QQ'
))
# Reference line y=x for comparison
min_val = min(theoretical_q1.min(), theoretical_q2.min())
max_val = max(theoretical_q1.max(), theoretical_q2.max())
fig_qq.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Reference Line'
))

fig_qq.update_layout(
    title="Q-Q Plot: Sample Distributions vs. Theoretical Normal",
    xaxis_title="Theoretical Quantiles",
    yaxis_title="Sample Quantiles"
)
fig_qq.show()